In [1]:
import re    # for regular expressions 
import nltk  # for text manipulation 
from nltk.corpus import stopwords
import string # for text manipulation 
import warnings
import pandas as pd # for data manipulation 
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore") #ignore warnings

# import ssl
# 
# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context
# 
# nltk.download()

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%matplotlib inline

./data/processed.csv
./data/light_data.csv
./data/light.csv
./data/data.csv
./data/processed
./data/lightdata.csv


In [2]:
data = pd.read_csv("./data/data.csv",encoding='latin-1')
data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [3]:
data = data.sample(frac=1)

In [4]:
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "TweetText"]
data.columns = DATASET_COLUMNS
data.head()

,target,ids,date,flag,user,TweetText
1178409,4,1981580293,Sun May 31 09:27:17 PDT 2009,NO_QUERY,3v3tT3,home from church...service was good
885235,4,1686449110,Sun May 03 05:29:04 PDT 2009,NO_QUERY,agriggs8,@DetroitSkye My son is 5 - It's weird to go from a 5 yr old to baby. It's certainly a change! All worth it though.
846239,4,1564365563,Mon Apr 20 03:08:06 PDT 2009,NO_QUERY,henasraf,@Lironster Mmmm I ate some yesterday.
683752,0,2250357719,Sat Jun 20 00:19:32 PDT 2009,NO_QUERY,HeatherSisemore,Has a very busy day tomorrow. So why am I not sleepy
661163,0,2243026452,Fri Jun 19 12:56:00 PDT 2009,NO_QUERY,deathblade228,"So i cleaned my car for 3 hours and it took forever, then driving to work about 1 min in a bird shits on my clean car"


In [5]:
data.drop(['ids','date','flag','user'],axis = 1,inplace = True)
data.head()

,target,TweetText
1178409,4,home from church...service was good
885235,4,@DetroitSkye My son is 5 - It's weird to go from a 5 yr old to baby. It's certainly a change! All worth it though.
846239,4,@Lironster Mmmm I ate some yesterday.
683752,0,Has a very busy day tomorrow. So why am I not sleepy
661163,0,"So i cleaned my car for 3 hours and it took forever, then driving to work about 1 min in a bird shits on my clean car"


In [ ]:
positif_data = data[data.target==4].iloc[:2500,:]
print(positif_data.shape)
negative_data = data[data.target==0].iloc[:100,:]
print(negative_data.shape)

In [6]:
stopwords_list=stopwords.words('english')
from nltk.stem.porter import *
stemmer = PorterStemmer()
def remove_stopwords(text):
    clean_text=' '.join([word for word in text.split() if word not in stopwords_list])
    return clean_text

In [7]:
# data = pd.concat([positif_data,negative_data],axis = 0)
data['Clean_TweetText'] = data['TweetText'].str.replace("@", "")
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace(r"http\S+", "")
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace("[^a-zA-Z]", " ")
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda text : remove_stopwords(text.lower()))
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: x.split())
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: [stemmer.stem(i) for i in x])
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x]))

In [8]:
data = data.drop(labels=['TweetText'], axis=1)

In [9]:
data.head()

,target,Clean_TweetText
1178409,4,home church...servic good
885235,4,detroitsky weird baby. certainli change! worth though.
846239,4,lironst mmmm yesterday.
683752,0,busi tomorrow. sleepi
661163,0,"clean hour took forever, drive work bird shit clean"


In [11]:
data.to_csv('./data/processed_full.csv', sep=",",quotechar='"',index=False)